email agent
- authenticates user
    - only then are they allowed into the "inbox"
    - dynamic tools and prompt on the condition of there being an email and password in state that match hardcoded
- checks "inbox"
    - email in tool
- sends emails
    - human in the loop

In [1]:
from dotenv import load_dotenv

load_dotenv()


True

In [2]:
from dataclasses import dataclass

@dataclass
class EmailContext:
    email_address: str = "julie@example.com"
    password: str = "password123"

In [3]:
from langchain.agents import AgentState

class AuthenticatedState(AgentState):
    authenticated: bool

In [4]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

@tool
def check_inbox() -> str:
    """Check the inbox for recent emails"""
    return """
    Hi Julie, 
    I'm going to be in town next week and was wondering if we could grab a coffee?
    - best, Jane (jane@example.com)
    """

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send an response email"""
    return f"Email sent to {to} with subject {subject} and body {body}"

@tool
def authenticate(email: str, password: str, runtime: ToolRuntime) -> Command:
    """Authenticate the user with the given email and password"""
    if email == runtime.context.email_address and password == runtime.context.password:
        return Command(update={
            "authenticated": True, 
            "messages": [ToolMessage(
                "Successfully authenticated", 
                tool_call_id=runtime.tool_call_id)]
        })
    else:
        return Command(update={
            "authenticated": False,
            "messages": [ToolMessage(
                "Authentication failed", 
                tool_call_id=runtime.tool_call_id)]
        })

In [5]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

@wrap_model_call
def dynamic_tool_call(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:

    """Allow read inbox and send email tools only if user provides correct email and password"""

    authenticated = request.state.get("authenticated")
    
    if authenticated:
        tools = [check_inbox, send_email]
    else:
        tools = [authenticate]

    request = request.override(tools=tools) 
    return handler(request)

In [6]:
from langchain.agents.middleware import dynamic_prompt

authenticated_prompt = "You are a helpful assistant that can check the inbox and send emails."
unauthenticated_prompt = "You are a helpful assistant that can authenticate users."

@dynamic_prompt
def dynamic_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on authentication status"""
    authenticated = request.state.get("authenticated")

    if authenticated:
        return authenticated_prompt
    else:
        return unauthenticated_prompt

In [7]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware

agent = create_agent(
    "gpt-5-nano",
    tools=[authenticate, check_inbox, send_email],
    checkpointer=InMemorySaver(),
    state_schema=AuthenticatedState,
    context_schema=EmailContext,
    middleware=[
        dynamic_tool_call, 
        dynamic_prompt,
        HumanInTheLoopMiddleware(
            interrupt_on={
                "authenticate": False,
                "check_inbox": False,
                "send_email": True,
            })
        ]
    )


In [8]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
            {"messages": [HumanMessage(content="Please check my inbox.")]},
            context = EmailContext(),
            config = config
)

print(response['messages'][-1].content)

Sure—I'll check your inbox. Please provide your login details to sign in:

- Email
- Password

(Your credentials will be used only to authenticate this session.)


In [9]:
response = agent.invoke(
            {"messages": [HumanMessage(content="julie@example.com, password123")]},
            context = EmailContext(),
            config = config
)

print(response['messages'][-1].content)

/Users/suvo/Documents/Langchain/lca-lc-foundations/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value=EmailContext(email_addres... password='password123'), input_type=EmailContext])
  return self.__pydantic_serializer__.to_python(


I found 1 new message from Jane (jane@example.com).

Message:
Hi Julie, 
I'm going to be in town next week and was wondering if we could grab a coffee?
- best, Jane (jane@example.com)

Would you like me to reply? I can send a draft like this, or customize with your preferred times.

Draft reply (subject: Re: Coffee next week):
Hi Jane,
That sounds great! I'd love to catch up. I'm free Tuesday afternoon or Thursday morning next week—do either of those work for you? If not, feel free to suggest another time and place.
Best,
Julie

If you want, I can:
- Send this draft now (or with your preferred times)
- Ask Jane for a specific time
- Archive or mark the message as read after replying
Tell me how you’d like to proceed.


In [10]:
# from langchain.messages import HumanMessage

# response = agent.invoke(
#     {"messages": [HumanMessage(content="Use Option A - Casual Response")]},
#     context=EmailContext(),
#     config=config
# )

# print(response['messages'][-1].content)

In [11]:
#print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

In [12]:
# from langgraph.types import Command

# response = agent.invoke(
#     Command(resume={"decisions": [{"type": "reject"}]}),
#     config=config
# )

# print(response["messages"][-1].content)

In [13]:
from langchain_core.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(
        content="Propose Tuesday 10:00 AM at Cannelle. Show the final email (To/Subject/Body) only. Do NOT send and do NOT call send_email."
    )]},
    config=config
)

print(response["messages"][-1].content)


To: Jane <jane@example.com>
Subject: Re: Coffee next week
Body:
Hi Jane,
That sounds great! I'd love to catch up. How does Tuesday at 10:00 AM at Cannelle work for you?
If that time doesn't work, feel free to suggest another time and place.
Best,
Julie


In [14]:
from pprint import pprint

pprint(response)

{'authenticated': True,
 'messages': [HumanMessage(content='Please check my inbox.', additional_kwargs={}, response_metadata={}, id='12ef009e-a4a4-4ca9-b4df-6d17b9d11aa7'),
              AIMessage(content="Sure—I'll check your inbox. Please provide your login details to sign in:\n\n- Email\n- Password\n\n(Your credentials will be used only to authenticate this session.)", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 619, 'prompt_tokens': 149, 'total_tokens': 768, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 576, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CqfKv1lJUq2ZaiXwRjinS3WFmWJLu', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b559e-32e1-7463-b4ac-260b221c142a-0', usage_met

In [15]:
from langchain_core.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="Reply to Jane and SEND the email now. Use the send_email tool.")]},
    config = config
)


In [16]:
pprint(response)

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': 'Hi '
                                                                          'Jane,\n'
                                                                          'That '
                                                                          'sounds '
                                                                          'great! '
                                                                          "I'd "
                                                                          'love '
                                                                          'to '
                                                                          'catch '
                                                                          'up. '
                                                                          'How '
                                                                          'does '
                

In [17]:
from langgraph.types import Command

response2 = agent.invoke(
    Command(resume={"decisions": [{"type": "approve"}]}),
    config=config
)

print(response2["messages"][-1].content)

All set. The email has been sent to Jane (jane@example.com) with subject "Re: Coffee next week".

Body that was sent:
Hi Jane,
That sounds great! I'd love to catch up. How does Tuesday at 10:00 AM at Cannelle work for you?
If that time doesn't work, feel free to suggest another time and place.
Best,
Julie

Would you like me to set a reminder for Tuesday at 10:00 AM, or archive the thread after Jane replies?


In [18]:
pprint(response2)

{'authenticated': True,
 'messages': [HumanMessage(content='Please check my inbox.', additional_kwargs={}, response_metadata={}, id='12ef009e-a4a4-4ca9-b4df-6d17b9d11aa7'),
              AIMessage(content="Sure—I'll check your inbox. Please provide your login details to sign in:\n\n- Email\n- Password\n\n(Your credentials will be used only to authenticate this session.)", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 619, 'prompt_tokens': 149, 'total_tokens': 768, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 576, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CqfKv1lJUq2ZaiXwRjinS3WFmWJLu', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b559e-32e1-7463-b4ac-260b221c142a-0', usage_met